In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pickle
from google.colab import drive
# drive.mount('/content/drive')
import os
import time

%matplotlib inline

debugging_mode = 0

In [ ]:
%load_ext autoreload
#using autoreload 1 to reload all modules imported by %aimport everytime before executing the python code typed .
%autoreload 1

%aimport lane

ModuleNotFoundError: ignored

In [ ]:
from google.colab.patches import cv2_imshow

vidloc = r"/content/drive/My Drive/test images/project_video.mp4"
w_path = r"/content/drive/My Drive/yolov3.weights"
c_path = r"/content/drive/My Drive/yolov3.cfg"

labels = open(r"/content/drive/My Drive/coco.names").read().strip().split("\n")

net = cv2.dnn.readNetFromDarknet(c_path, w_path)
names = net.getLayerNames()
layers_names = [names[i[0]-1] for i in net.getUnconnectedOutLayers()]

cap = cv2.VideoCapture(vidloc)
W = int(cap.get(3))
H = int(cap.get(4))
fps=int(cap.get(5))
count=0
fcount=0
out_stream=[]
while cap.isOpened():
    ret, frame = cap.read()
   
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    final = lane.vid_pipeline(frame)
    # final=frame
    count=count+1
    fcount=fcount+1
    time1=time.time()
    if(count==1):
      blob = cv2.dnn.blobFromImage(final, 1/255.0, (416,416), crop=False, swapRB = False)
      net.setInput(blob)
      layers_output = net.forward(layers_names)
      # print("image into network time is",time.time()-time1)
      boxes=[]
      confidences=[]
      classIDs=[]

      # time1=time.time()
      for output in layers_output:
        for detection in output:
          scores=detection[5:]
          classID = np.argmax(scores)
          confidence = scores[classID]

          if(confidence>0.85):
            box=detection [:4] * np.array([W, H, W, H])
            bx, by, bw, bh = box.astype("int")
            Px= int(bx-(bw/2))
            Py= int(by-(bh/2))
            boxes.append([Px,Py,int(bw),int(bh)])
            confidences.append(float(confidence))
            classIDs.append(classID)

    idxs= cv2.dnn.NMSBoxes(boxes,confidences,0.8,0.8)
    # print("extracting data time is ",time.time()-time1)
    # print(idxs)

    # time1=time.time()
    if(len(idxs)>0):
        for i in idxs.flatten():
          (x,y) = [boxes[i][0], boxes[i][1]]
          (w,h) = [boxes[i][2],boxes[i][3]]

          cv2.rectangle(final,(x,y),(x+w,y+h),(200,100,0),2)
          cv2.putText(final,"{} : {}%".format(labels[classIDs[i]],int(confidences[i]*100)),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(200,100,0),2)
    # print("Printing data time is ")
    
    # count=count+1
    if(count==10):
      count=0
    final =cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
    out_stream.append(final)
    # cv2_imshow(final)
    print("frame {} is done".format(fcount),time.time()-time1)

    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('video_c.avi', fourcc, 25, (W,H))

for j in range(len(out_stream)-1):
  img = out_stream[j]
  video.write(img)
  
cv2.destroyAllWindows()
video.release()





In [ ]:
from google.colab.patches import cv2_imshow

vidloc = r"/content/drive/My Drive/test images/challenge_video.avi"

cap = cv2.VideoCapture(vidloc)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps=int(cap.get(5))
print(fps)
count=0
out_stream=[]
while cap.isOpened():
    ret, frame = cap.read()
   
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    final = lane.vid_pipeline(frame)
    count=count+1
    print("frame",count)
    out_stream.append(final)
    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('video_c.avi', fourcc, 25, (frame_width,frame_height))

for j in range(len(out_stream)-1):
  img = out_stream[j]
  video.write(img)
  
cv2.destroyAllWindows()
video.release()

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import math
import copy as c


def sub(pt1,pt2):
  diffs=[]
  diffs.append(abs(pt1[0]-pt2[0]))
  diffs.append(abs(pt1[1]-pt2[1]))
  # print(diffs)

  return max(diffs)




def ver(line):
  if(abs(line[0][0]-line[1][0])-abs(line[0][1]-line[1][1])>0):
    return 0
  else:
    return 1

def count_ver(set_lines):
  verc=0
  horc=0
  for sl in set_lines:
    if(len(sl)>0):
      l=sl[0]
      # print(l)
      if(ver(l)==1):
        verc+=1
      else:
        horc+=1
  return(verc,horc)
  




def avg_line(sl):
  
  if(ver(sl[0])==1):
    x1=[]
    x2=[]
    for l in sl:
      x1.append(l[0][0])
      x2.append(l[1][0])
    x1=int(sum(x1)/len(x1))
    y1=int(sl[0][0][1])
    x2=int(sum(x2)/len(x2))
    y2=int(sl[0][1][1])





  if(ver(sl[0])==0):
    y1=[]
    y2=[]
    for l in sl:
      y1.append(l[0][1])
      y2.append(l[1][1])
    x1=int(sl[0][0][0])
    y1=int(sum(y1)/len(y1))
    y2=int(sum(y2)/len(y2))
    x2=int(sl[0][1][0])


  return ((x1,y1),(x2,y2))




def avg_points(points):
  p1=[]
  p2=[]
  p3=[]
  p4=[]
  p1.append(points[0])
  for i in range(1,len(points)):
    p=points[i]
    if(len(p1)>0):
      if((abs(p[0]-p1[0][0])<10)&(abs(p[1]-p1[0][1])<10)):
        p1.append(p)
      else:
        if(len(p2)>0):
          if((abs(p[0]-p2[0][0])<10)&(abs(p[1]-p2[0][1])<10)):
            p2.append(p)
          else:
            if(len(p3)>0):
              if((abs(p[0]-p3[0][0])<10)&(abs(p[1]-p3[0][1])<10)):
                p3.append(p)
              else:
                if(len(p4)>0):
                  if((abs(p[0]-p4[0][0])<10)&(abs(p[1]-p4[0][1])<10)):
                    p4.append(p)
                else:
                   p4.append(p)
            else:
              p3.append(p)
        else:
          p2.append(p)
    else:
      p1.append(p)             

  p1=p1[0]
  p2=p2[0]
  p3=p3[0]
  p4=p4[0]
  return[p1,p2,p3,p4]





def distance(points):
  distances=[]
  for i in range(len(points)):
    for j in range(len(points)-i-1):
      p1=points[i]
      p2=points[j+i+1]
      distance=math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2))
      distances.append(distance)
  return distances











loc = r"/content/WhatsApp Image 2022-06-06 at 7.58.27 PM (1).jpeg"
img = cv2.imread(loc)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
rows,cols,_ = img.shape

for i in range(rows):
    for j in range(cols):
        if((i>int(3*rows/5))|(i<int(rows/10))|(j<(int(5*cols/18)))|(j>int(2*cols/3))):
          gray[i,j]=0
        else:
          gray[i,j]=gray[i,j]

for i in range(rows):
    for j in range(cols):
        if(gray[i,j]>60):
          gray[i,j]=250
        else:
          gray[i,j]=20

init_k=31
dists=[]
while(1):
  kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(init_k,init_k))
  gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
  # cv2_imshow(gray)
  dst_img = cv2.Canny(gray, 50, 200, None, 3)
  lines=cv2.HoughLines(dst_img, 1, np.pi / 180, 40, None, 0, 0)



  line_set=[[],[]]
  ref_line=lines[0][0]
  # print(ref_line)
  ref_thet=0
  ref_thet2=ref_thet+np.pi
  perp_thet=ref_thet+(np.pi/2)
  perp_thet2=perp_thet-np.pi
  th_e=0.3
  ro_e=math.sin(th_e)
  for line in lines:
    tang=0
    perp=0
    rho=line[0][0]
    thet=line[0][1]
    d="inf"
    if((abs(ref_thet-thet)<=th_e)|(abs(ref_thet2-thet)<=th_e)):
      line_set[0].append(line[0])
    if((abs(perp_thet-thet)<=th_e)|(abs(perp_thet2-thet)<=th_e)):
      line_set[1].append(line[0])
    




  # print(lines)
  set_lines=[[],[],[],[]]
  rho_l = lines[0][0][0]
  theta_l = lines[0][0][1]
  a_l = math.cos(theta_l)
  b_l = math.sin(theta_l)
  x0_l = a_l * rho_l
  y0_l = b_l * rho_l
  pt1_l = (int(x0_l + 600*(-b_l)), int(y0_l + 600*(a_l)))
  pt2_l = (int(x0_l - 600*(-b_l)), int(y0_l - 600*(a_l)))
  ref_l=(pt1_l,pt2_l)
  set_lines[0].append(ref_l)
  # cv2.line(gray, pt1_l, pt2_l, (155,150,155), 1, cv2.LINE_AA)
  # cv2_imshow(gray)
  for i in range(1, len(lines)):
              # print("line",lines[i])
              rho_l = lines[i][0][0]
              theta_l = lines[i][0][1]
              a_l = math.cos(theta_l)
              b_l = math.sin(theta_l)
              x0_l = a_l * rho_l
              y0_l = b_l * rho_l
              pt1_l = (int(x0_l + 600*(-b_l)), int(y0_l + 600*(a_l)))
              pt2_l = (int(x0_l - 600*(-b_l)), int(y0_l - 600*(a_l)))
              l=(pt1_l,pt2_l)
              appended=0
              # print("l_points",l)
              for lset in set_lines:
                if(len(lset)>0):
                  avg=np.average(lset,axis=0)
                
                  # print("avg",avg)
                  # print("point of measure:",lset[0]," pt1",pt1_l," pt2",pt2_l," diff",sub(pt2_l,lset[0][0]),sub(pt2_l,lset[0][1]),sub(pt1_l,lset[0][1]),sub(pt2_l,lset[0][0]))
                  # (sub(pt1_l,lset[0][0])<40)&(sub(pt2_l,lset[0][1])<40))|((sub(pt1_l,lset[0][1])<40)&(sub(pt2_l,lset[0][0])<40)
                  # print("set ",set_lines)
                  if(((sub(pt1_l,lset[0][0])<40)&(sub(pt2_l,lset[0][1])<40))|((sub(pt1_l,lset[0][1])<40)&(sub(pt2_l,lset[0][0])<40))):
                    if(ver(l)==ver(lset[0])):
                      lset.append(l)
                      appended=1
                  if(appended):
                    break
                else:
                  (verc,horc)=count_ver(set_lines)
                
                  if(((ver(l)==1)&(verc<2))|((ver(l)==0)&(horc<2))):
                  
                    lset.append(l)
                    break
                # cv2.line(gray, pt1_l, pt2_l, (155,150,155), 1, cv2.LINE_AA)
                # cv2_imshow(gray)


  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)





  black=c.deepcopy(gray)
  for i in range(rows):
      for j in range(cols):
          black[i,j]=0

  # cv2_imshow(black)
  true_lines=[]
  for set_l in set_lines:
    true_lines.append(avg_line(set_l))
  # print(true_lines)
  # for set_l in set_lines:
  for l in true_lines:
    pt1_l=l[0]
    pt2_l=l[1]
    cv2.line(black, pt1_l, pt2_l, (155,150,155), 1, cv2.LINE_AA)
    # cv2_imshow(black)

  kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

  # kernel[0][0]=0
  black=cv2.erode(black,kernel)
  points=[]
  for i in range(rows):
      for j in range(cols):
          if(black[i,j]>50):
            points.append((i,j))

  # print(points)






  
  try:
    points= avg_points(points)
  except:
    init_k=+2
    continue
  dists=distance(points)
  dists.sort()
  dists=dists[0:4]
  for i in range(len(dists)):
    dists[i]=(dists[i]/44.5)+(18/44.5)
  break











dists   



[3.4614176227226516, 3.4681006925337985, 3.6180561002415934, 3.9775280898876404]


In [ ]:
l=((595, 95), (602, 32))
ver(l)

1

In [ ]:
cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))


array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [ ]:
from google.colab.patches import cv2_imshow

img = cv2.imread(r"/content/city_AlexanderSpatari_Getty-5a13972b47c26600378f0216.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# dst = lane.combine_thresh(img)
# dst = lane.perspective_warp(dst)
cv2_imshow(img)
filter=[[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]

                    
histo1 = plt.hist(img.ravel(),256)
plt.show()
# Apply Laplacian operator in some higher datatype
img2 = cv2.Laplacian(img,cv2.CV_32F)
img3=cv2.Laplacian(img,cv2.CV_64F)
cv2_imshow(img2)
cv2_imshow(img3)
histo2 = plt.hist(img2.ravel(),256)
plt.show()

# x_hist = np.array(range(0, 1280))
# plt.plot(x_hist, histo)
print("\n\n\n\n")
# histo1

# histo2

img3=img+img2
cv2_imshow(img3)
histo1 = plt.hist(img3.ravel(),256)
plt.show()

# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
# ax1.imshow(histo1, cmap="gray")
# ax1.set_title('Original Image', fontsize=30)
# ax2.imshow(histo2, cmap='gray')
# ax2.set_title('Warp', fontsize=30)

In [ ]:
from google.colab.patches import cv2_imshow
loc = r"/content/drive/My Drive/test images/test2.jpg"
img = cv2.imread(loc)
cv2_imshow(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

dst = lane.dir_threshold(img,thresh=(np.pi/6,np.pi/2))
cv2_imshow(dst)
# out_img, curves, lanes, ploty = lane.sliding_window(dst)
# # plt.imshow(dst)
# plt.plot(curves[0], ploty, color?='blue', linewidth=3)
# plt.plot(curves[1], ploty, color='blue', linewidth=3)

# curverad= lane.get_curve(img, curves[0],curves[1])

# img_ = lane.draw_lanes(img, curves[0], curves[1])
# plt.imshow(out_img, cmap='hsv')